# Flower Detect
Identify and classify flower types based on their images.

In [1]:
data_dir=r"D:\Data centr\IMG_data\flowers"

# Dataset Collection
* Augment Data- Rotation, Croping, etc

Batch size is the number of training sample used in one forward and backward pass through the model during training.

Sample= 1,000, batch_size = 32,
At each iteration,process 32 samples.

Iteration= 1000 ÷ 32 = ~32.

In [2]:
img_hgt=224
img_wdt=224
batch_sze=32 # 32 Sample

# Preprocess the Data
Build a CNN
* Resize all images to a fixed size (224x224)
* Normalize pixel values (0–1)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [4]:
train_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Preprocessing for MobileNetV2
    rotation_range=30, # rotate image by 30 degree
    width_shift_range=0.2,  # shift horizontally by 20% of width
    height_shift_range=0.2,  # shift vertically by 20% of height
    shear_range=0.2, # Shearing skews the image along one axis, making it look like the image is "slanted."
    zoom_range=0.2, # zooms in or out by up to 20%
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2 # Split for training and validation set by 20% of images
)

# Split the dataset
Train_data & Validation_data

In [5]:
train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_hgt, img_wdt),
    batch_size=batch_sze,
    class_mode='categorical',
    subset='training'
)

Found 604 images belonging to 11 classes.


In [6]:
val_data=train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_hgt,img_wdt),
    batch_size=batch_sze,
    class_mode="categorical",
    subset='validation'
)

Found 145 images belonging to 11 classes.


# Train the Model
* Loss Function: Use categorical_crossentropy for multi-class classification.
* Optimizer: Use Adam with a learning rate of 0.001.
* Epochs: The number of times the entire dataset is processed during training.
Start with 20–50 epochs, depending on the dataset size.

In [7]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import tensorflow as tf


In [8]:
base_model=MobileNetV2( # pre trained deep learning model
    weights='imagenet', # trained on the ImageNet dataset (1,000 categories like dogs, cats, cars, etc)
    include_top=False, # Removes the final classification layer of MobileNetV2 and add custom layers for flower dataset.
    input_shape=(img_hgt,img_wdt,3) # 3- color channer(RGB)
    )
base_model.trainable=False  # Freezing the base model means the weights of MobileNetV2 will not be updated during training.

In [9]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)  # Fully connected layer
output_tensor = Dense(5, activation='softmax')(x)  # Output layer for 5 classes
x = Dense(128, activation='relu')(x)
output_tensor = Dense(5, activation='softmax')(x)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [11]:
model=Sequential([
    base_model, #frozen MobileNetV2 model, which acts as a feature extractor (e.g., shapes, textures, etc).
    tf.keras.layers.GlobalAveragePooling2D(), # Converts a 2D feature map into a 1D feature vector.
    Dense(128,activation='relu'), # Fully connected layer with 128 neurons and ReLU activation.
    Dropout(0.3), # Prevents overfitting by randomly setting 30% of the neurons to 0 during training
    Dense(train_data.num_classes,activation='softmax') # Output layer for flower types
])

Adam is an adaptive optimizer that adjusts learning rates during training for better convergence.

Categorical_crossentropy: Measures the difference between the predicted probability distribution (output of softmax) and the true labels.


In [12]:
from tensorflow.keras.optimizers import Adam 

In [13]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

# CNN Training

In [14]:
epochs=10
history=model.fit(train_data,validation_data=val_data,epochs=epochs)

c:\Users\himan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 822ms/step - accuracy: 0.2551 - loss: 2.3559

c:\Users\himan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.2638 - loss: 2.3301 - val_accuracy: 0.7103 - val_loss: 0.9261
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.7489 - loss: 0.8097 - val_accuracy: 0.8690 - val_loss: 0.5530
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.8285 - loss: 0.5349 - val_accuracy: 0.8966 - val_loss: 0.3799
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8605 - loss: 0.4128 - val_accuracy: 0.8828 - val_loss: 0.3904
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9071 - loss: 0.3126 - val_accuracy: 0.8552 - val_loss: 0.4625
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9026 - loss: 0.3242 - val_accuracy: 0.8897 - val_loss: 0.3704
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.9308 - loss: 0.2307 - val_accuracy: 0.9379 - val_loss: 0.2993
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9418 - loss: 0.2092 - val_accuracy: 0.9172 - val_loss: 0.3011
Epo

# Evaluate the Model
* Use accuracy, precision, recall, and F1-score to measure performance.
* Plot training and validation loss/accuracy to ensure no overfitting.

In [15]:
val_loss,val_accuracy=model.evaluate(val_data)
print(f"validation Accuracy:{val_accuracy*100:.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 732ms/step - accuracy: 0.9160 - loss: 0.3132
validation Accuracy:91.72%


# Class Mapping

In [16]:
model.save('flower_classifier_model.h5')
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,754,147 (10.51 MB)

 Trainable params: 165,387 (646.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

 Optimizer params: 330,776 (1.26 MB)

None


In [17]:
class_name=list(train_data.class_indices.keys()) # A dictionary mapping class names to integer labels list.
print("Class Label:",class_name)

Class Label: ['HRaj', 'bougainvillea', 'daisies', 'garden_rose', 'gardenias', 'hibiscus', 'hydrangeas', 'lilies', 'orchids', 'peonies', 'tulip']


# Testing Model

In [18]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [19]:
def predict_flower(image_path):
    # Load and preprocess the image
    img=image.load_img(image_path,target_size=(img_hgt,img_wdt))
    img_array=image.img_to_array(img)
    img_array=np.expand_dims(img_array,axis=0)
    img_array=preprocess_input(img_array)

    # Predict the flower type
    prediction=model.predict(img_array)
    prediction_class=class_name[np.argmax(prediction)]
    confidence=np.max(prediction)


    """    
    # Display the image with prediction
    plt.figure(figsize=(6, 6))
    plt.imshow(plt.imread(image_path))  # Load the image for display
    plt.axis('off')  # Remove axis labels
    plt.title(f"Predicted: {prediction_class}\nConfidence: {confidence * 100:.2f}%")
    plt.show()
    """
    
    return prediction_class,confidence

# Normalization

In [20]:
from tensorflow.keras.layers import Rescaling

In [21]:
model_preprocessed=tf.keras.Sequential([
    Rescaling(1./255, input_shape=(224,224,3)), # Normalize layer
    model
])

c:\Users\himan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


File Saving

In [22]:
converter=tf.lite.TFLiteConverter.from_keras_model(model_preprocessed)
tflite_model=converter.convert()

with open('flower_classifier_model.tflite','wb') as f:
    f.write(tflite_model)
print(help(tf.lite.TFLiteConverter))

INFO:tensorflow:Assets written to: C:\Users\himan\AppData\Local\Temp\tmpe3rgli5i\assets


INFO:tensorflow:Assets written to: C:\Users\himan\AppData\Local\Temp\tmpe3rgli5i\assets


Saved artifact at 'C:\Users\himan\AppData\Local\Temp\tmpe3rgli5i'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_165')
Output Type:
  TensorSpec(shape=(None, 11), dtype=tf.float32, name=None)
Captures:
  2478362558928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2478362561808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2478362561232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2478362561424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2478362562192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2478362562384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2478362559888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2478362561616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2478362560272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2478362562768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  

In [1]:
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils


ModuleNotFoundError: No module named 'tflite_support'

In [26]:
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils

# Paths
model_path = "flower_classifier_model.tflite"
export_path = "flower_classifier_with_metadata.tflite"

# Image metadata
input_norm_mean = 127.5
input_norm_std = 127.5
label_file_path = "labels.txt"  # Ensure this file contains class names, one per line

# Create metadata writer
writer = image_classifier.MetadataWriter.create_for_inference(
    writer_utils.load_file(model_path),
    input_norm_mean=input_norm_mean,
    input_norm_std=input_norm_std,
    label_file=label_file_path
)

# Write metadata to file
writer_utils.save_file(writer.populate(), export_path)
print(f"Metadata added to {export_path}")


ModuleNotFoundError: No module named 'tflite_support'

In [23]:
img_pth=r"D:\Data centr\IMG_data\flowers\hydrangeas\hydrangeas_00010.jpg"

# Prediction Result

In [24]:
flower_type,confidence=predict_flower(img_pth)
print(f"Predicted Flower Type: {flower_type}, Confidence: {confidence * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Flower Type: hydrangeas, Confidence: 99.55%
